In [1]:
cd /mnt/c/Users/nakagawa/Desktop/FEM4INAS/FEM4INAS/examples/BUG/PARAMETRIC/

/mnt/c/Users/nakagawa/Desktop/FEM4INAS/FEM4INAS/examples/BUG/PARAMETRIC


/mnt/c/Users/nakagawa/Desktop/python_venv/feniax/lib/python3.12/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [2]:
import jax
jax.config.update("jax_enable_x64", True)
import os,sys
sys.path.append('/mnt/c/Users/nakagawa/Desktop/FEM4INAS/FEM4INAS/examples/BUG/PARAMETRIC/')
from importlib import reload
import sensitivity_handler
import nastran_preprocessor
import bug_handler
import shutil
import nastran_tools

In [3]:
reload(sensitivity_handler)
reload(nastran_preprocessor)
reload(bug_handler)
reload(nastran_tools)
from sensitivity_handler import *
from nastran_preprocessor import *
from bug_handler import *
from nastran_tools import *

In [8]:
from parametric_study import decode_params

In [4]:
import os
os.environ["XLA_PYTHON_CLIENT_PREALLOCATE"] = "false"
os.environ["XLA_PYTHON_CLIENT_ALLOCATOR"]="platform"

In [5]:
bdfname='./BUG_103cao.bdf'
handler=BUGHandler(bdfname)
for name in AnnotationLabels:
  fname='./Parts/'+name+'.bdf'
  label=(name.split('_')[1]).upper()
  handler.add_annotation(fname,label)
handler.add_caero_annotation()
handler.add_pshell_annotation()
_=handler.get_component_names()

In [6]:
nastran_loc='cmd.exe /c C:/MSC.Software/MSC_Nastran/20182/bin/nast20182.exe'

In [7]:
params=dict()
params['WING_SKIN-FEM-THICKNESS']=np.array([1.0,1.0])

In [9]:
params_v_raw=dict()
params_v_raw['WING_SKIN-FEM-THICKNESS']=[[0.8,1.0,1.2],[0.8,1.0,1.2]]
params_v=decode_params(params_v_raw)

In [10]:
params_v

[{'WING_SKIN-FEM-THICKNESS': [0.8, 0.8]},
 {'WING_SKIN-FEM-THICKNESS': [0.8, 1.0]},
 {'WING_SKIN-FEM-THICKNESS': [0.8, 1.2]},
 {'WING_SKIN-FEM-THICKNESS': [1.0, 0.8]},
 {'WING_SKIN-FEM-THICKNESS': [1.0, 1.0]},
 {'WING_SKIN-FEM-THICKNESS': [1.0, 1.2]},
 {'WING_SKIN-FEM-THICKNESS': [1.2, 0.8]},
 {'WING_SKIN-FEM-THICKNESS': [1.2, 1.0]},
 {'WING_SKIN-FEM-THICKNESS': [1.2, 1.2]}]

In [ ]:
preprocessor=NastranPreprocessor(nastran_loc)
#variables: from root to tip
fname_grid='./FEM/structuralGrid'
preprocessor.set_config(bdfname,working_dir='./_temp_p',num_parallel=5)
preprocessor.parametric_study(params_v,fname_grid,None)